In [1]:
!pip install pandas numpy scipy kadlu kaleido==0.2.1 matplotlib pyarrow pooch seaborn plotly xarray ipywidgets cfgrib cartopy scikit-learn tensorflow

In [2]:
import pandas as pd                 # pandas is a dataframe library
import matplotlib.pyplot as plt      # matplotlib.pyplot plots data
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

%matplotlib inline

In [3]:
df = pd.read_parquet("./oasis_07_25.parquet")

In [4]:
df.head()

,datetime,hour_index,load_MW
datetime,,,
2007-01-01 00:00:00,2007-01-01 00:00:00,1,1580.0
2007-01-01 01:00:00,2007-01-01 01:00:00,2,1533.0
2007-01-01 02:00:00,2007-01-01 02:00:00,3,1495.0
2007-01-01 03:00:00,2007-01-01 03:00:00,4,1446.0
2007-01-01 04:00:00,2007-01-01 04:00:00,5,1406.0


In [5]:
df['hour_index']

datetime
2007-01-01 00:00:00       1
2007-01-01 01:00:00       2
2007-01-01 02:00:00       3
2007-01-01 03:00:00       4
2007-01-01 04:00:00       5
                       ... 
2025-12-31 19:00:00    8755
2025-12-31 20:00:00    8756
2025-12-31 21:00:00    8757
2025-12-31 22:00:00    8758
2025-12-31 23:00:00    8759
Name: hour_index, Length: 166557, dtype: int64

In [6]:
df2 = df[pd.notnull(df['load_MW'])].copy()
df2.shape

(159956, 3)

In [7]:
df2["datetime"] = pd.to_datetime(df2["datetime"])
#, "hour_index"
features = ["year", "month", "day", "hour", "weekday", "weekofyear", "quarter"]
df2[features] = df2.apply(lambda row: pd.Series({"year":row.datetime.year, "month":row.datetime.month, "day":row.datetime.day, "hour":row.datetime.hour, "weekday":row.datetime.weekday(), "weekofyear":row.datetime.weekofyear, "quarter":row.datetime.quarter }), axis=1)

X = df2[features]
y = df2[["load_MW"]]
split_test_size = 0.05

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_test_size, random_state=42) 
                            # test_size = 0.3 is 30%, 42 is the answer to everything

In [8]:
print("{0:0.2f}% in training set".format((len(X_train)/len(df2.index)) * 100))
print("{0:0.2f}% in test set".format((len(X_test)/len(df2.index)) * 100))

95.00% in training set
5.00% in test set


In [9]:
print("# rows in dataframe {0}".format(len(df2)))

# rows in dataframe 159956


In [10]:
#model = LinearRegression()
#model = MLPRegressor()
model = RandomForestRegressor()
#model.fit(X_train, y_train)
model.fit(X_train, y_train.values.ravel())

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [11]:
y_pred = model.predict(X_test)
print(mean_squared_error(y_test, y_pred))

1993.662149530743


In [12]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

print(f"Mean Absolute Error: {r2_score(y_test, y_pred)}")

Mean Squared Error: 1993.662149530743
Mean Absolute Error: 29.008552546469954
Mean Absolute Error: 0.9704694889241829


In [13]:
print(y_test[:5])

                     load_MW
datetime                    
2009-01-26 16:00:00  1893.00
2021-03-29 19:00:00  1400.79
2018-12-08 19:00:00  1901.18
2023-10-05 13:00:00  1153.68
2024-04-30 15:00:00  1150.16


In [14]:
dfpredict = df[pd.isnull(df['load_MW'])].copy()
dfpredict.shape

(6601, 3)

In [15]:
# Identify rows with missing 'Target' values
missing_target_mask = df['load_MW'].isnull()

In [16]:
missing_target_mask.head()

datetime
2007-01-01 00:00:00    False
2007-01-01 01:00:00    False
2007-01-01 02:00:00    False
2007-01-01 03:00:00    False
2007-01-01 04:00:00    False
Name: load_MW, dtype: bool

In [17]:
dfpredict["datetime"] = pd.to_datetime(dfpredict["datetime"])
features = ["year", "month", "day", "hour", "weekday", "weekofyear", "quarter"]
dfpredict[features] = dfpredict.apply(lambda row: pd.Series({"year":row.datetime.year, "month":row.datetime.month, "day":row.datetime.day, "hour":row.datetime.hour, "weekday":row.datetime.weekday(), "weekofyear":row.datetime.weekofyear, "quarter":row.datetime.quarter }), axis=1)

X_predict = dfpredict[features]
y_predictions = dfpredict[["load_MW"]]

In [18]:
predictions = model.predict(X_predict)

In [19]:
print(predictions)

[1618.0646 1330.85   1325.2767 ... 1497.7043 1470.0097 1427.6321]


In [20]:
predictions.shape

(6601,)

In [21]:
df.loc[missing_target_mask, 'load_MW'] = predictions

In [22]:
df.tail()

,datetime,hour_index,load_MW
datetime,,,
2025-12-31 19:00:00,2025-12-31 19:00:00,8755,1566.9004
2025-12-31 20:00:00,2025-12-31 20:00:00,8756,1534.5305
2025-12-31 21:00:00,2025-12-31 21:00:00,8757,1497.7043
2025-12-31 22:00:00,2025-12-31 22:00:00,8758,1470.0097
2025-12-31 23:00:00,2025-12-31 23:00:00,8759,1427.6321


In [23]:
df['load_MW'] = np.trunc(df['load_MW'] * 100) / 100

In [24]:
df.tail()

,datetime,hour_index,load_MW
datetime,,,
2025-12-31 19:00:00,2025-12-31 19:00:00,8755,1566.90
2025-12-31 20:00:00,2025-12-31 20:00:00,8756,1534.53
2025-12-31 21:00:00,2025-12-31 21:00:00,8757,1497.70
2025-12-31 22:00:00,2025-12-31 22:00:00,8758,1470.00
2025-12-31 23:00:00,2025-12-31 23:00:00,8759,1427.63


In [25]:
df.to_parquet('./oasis_07_25_ML.parquet', compression=None)